In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import torch
import sys
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# training parameters
epochs = 10
batch_size = 512*25

In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
# all the features in avazu dataset are sparse features
sparse_features = ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
                   'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
                   'device_model', 'device_type', 'device_conn_type',  # 'device_ip', 
                   'C14',
                   'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', ]
print('len(sparse_features)', len(sparse_features))  # id, click, device_ip, and day are not used. 25-4=21

target = ['click']

try:
    # read data from pkl directly
    data=pd.read_pickle('data_avazu.pkl')
    print('read_pickle ok')
except:    
    print('preprocess data and save it by pickle')
    data = pd.read_csv('train.csv')
    print('read train ok')
    data['day'] = data['hour'].progress_apply(lambda x: str(x)[4:6])
    data['hour'] = data['hour'].progress_apply(lambda x: str(x)[6:])
    # data = pd.read_csv('avazu_first_3d.csv',nrows=50)  # for test
    data[sparse_features] = data[sparse_features].fillna('-1', )
    # 1.Label Encoding for sparse features,and do simple Transformation for dense features
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])

    data.to_pickle('data_avazu.pkl')
    print('to_pickle ok')

print(data[:5])
print(data['day'].unique())

len(sparse_features) 21
read_pickle ok
             id  click  hour  C1  banner_pos  site_id  site_domain  \
0  1.000009e+18      0     0   2           0      582         7339   
1  1.000017e+19      0     0   2           0      582         7339   
2  1.000037e+19      0     0   2           0      582         7339   
3  1.000064e+19      0     0   2           0      582         7339   
4  1.000068e+19      0     0   2           1     4695         4456   

   site_category  app_id  app_domain  ...  device_conn_type  C14 C15  C16  \
0              2    7884         254  ...                 1  285   3    2   
1              2    7884         254  ...                 0  283   3    2   
2              2    7884         254  ...                 0  283   3    2   
3              2    7884         254  ...                 0  285   3    2   
4              0    7884         254  ...                 0  590   3    2   

   C17  C18  C19  C20  C21  day  
0   56    0    2    0   22   21  
1   56   

In [5]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=8)
                          for feat in sparse_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

print('split data. use the first 2 days for training')
print('the 3rd day data: 20% for validation, 80% for test')
train = data[data['day'] != '23']  # first 3 days: 21 22 23
test = data[data['day'] == '23']
val, test = train_test_split(test, test_size=0.8,random_state=2020)

train_model_input = {name: train[name] for name in feature_names}
val_model_input = {name: val[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

split data. use the first 2 days for training
the 3rd day data: 20% for validation, 80% for test


In [10]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = FiBiNET(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                bilinear_type='each',
                reduction_ratio=8, dnn_hidden_units=(400, 400, 400), l2_reg_linear=1e-4,
                l2_reg_embedding=1e-4, l2_reg_dnn=1e-4, init_std=1e-15, seed=1024, dnn_dropout=0.3,
                dnn_activation='relu',
                task='binary', device=device)
print('model', model)

cuda ready...
model FiBiNET(
  (embedding_dict): ModuleDict(
    (C1): Embedding(7, 8)
    (C14): Embedding(2626, 8)
    (C15): Embedding(8, 8)
    (C16): Embedding(9, 8)
    (C17): Embedding(435, 8)
    (C18): Embedding(4, 8)
    (C19): Embedding(68, 8)
    (C20): Embedding(172, 8)
    (C21): Embedding(60, 8)
    (app_category): Embedding(36, 8)
    (app_domain): Embedding(559, 8)
    (app_id): Embedding(8552, 8)
    (banner_pos): Embedding(7, 8)
    (device_conn_type): Embedding(4, 8)
    (device_id): Embedding(2686408, 8)
    (device_model): Embedding(8251, 8)
    (device_type): Embedding(5, 8)
    (hour): Embedding(24, 8)
    (site_category): Embedding(26, 8)
    (site_domain): Embedding(7745, 8)
    (site_id): Embedding(4737, 8)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (C1): Embedding(7, 1)
      (C14): Embedding(2626, 1)
      (C15): Embedding(8, 1)
      (C16): Embedding(9, 1)
      (C17): Embedding(435, 1)
      (C18): Embedding(4, 1)
      (C19): E

In [15]:
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

In [18]:
def get_metrics(groups) -> pd.DataFrame:
    metrics =  pd.DataFrame.from_records(groups, columns=groups.keys(), index=list(all_metrics.values())[0].keys())
    metrics.index.name = 'metric'
    return metrics

def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    metrics = get_metrics(groups)
    return metrics.subtract(metrics[control_name], axis='index').drop(columns=control_name) * 100


all_metrics = {}

In [8]:
def test_model(model, train_model_input, test_model_input, val_model_input, y_train, y_test, y_valid, batch_size=4096 * 2):
    model_name = model.__class__.__name__
    
    model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"])
    
#     es = EarlyStopping(monitor='val_auc', min_delta=0, verbose=1, patience=0, mode='max')
#     mdckpt = ModelCheckpoint(filepath = f'{model_name}.ckpt', monitor='val_auc', mode='max', save_best_only= True)
    
    for epoch in range(1):
        print('epoch',epoch)
        model.fit(train_model_input, y_train,
                  batch_size=batch_size, epochs=1, validation_data=(val_model_input, y_valid), verbose=1)

        pred_ans = model.predict(test_model_input, batch_size*20)
        test_logloss = round(log_loss(y_test, pred_ans), 4)
        test_auc = round(roc_auc_score(y_test, pred_ans), 4)
        print("test LogLoss", test_logloss)
        print("test AUC", test_auc)
    
    
#     history = model.fit(train_model_input, y_train,
#                         batch_size=batch_size, epochs=15, verbose=2, validation_split=0.2, callbacks=[es, mdckpt])
    
#     model = torch.load(f'{model_name}.ckpt')
    
#     y_valid_predicted = model.predict(test_model_input, batch_size=batch_size).squeeze()
    torch.save(model, f'{model_name}.ckpt')
    return {
        'roc-auc': test_auc,
        'log-loss': test_logloss      
    }

In [19]:
all_metrics['FibiNet'] = test_model(model, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size)


epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [24:35,  1.94it/s]


Epoch 1/1
1485s - loss:  0.3883 - binary_crossentropy:  0.3883 - auc:  0.7625 - val_binary_crossentropy:  0.4159 - val_auc:  0.7432
test LogLoss 0.4147
test AUC 0.7437
epoch 1


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [24:38,  1.93it/s]


Epoch 1/1
1488s - loss:  0.3645 - binary_crossentropy:  0.3645 - auc:  0.7978 - val_binary_crossentropy:  0.4206 - val_auc:  0.7390
test LogLoss 0.4194
test AUC 0.7392


/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FiBiNET. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleDict. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type 

In [8]:
model.compile("adam", "binary_crossentropy",
              # metrics=["binary_crossentropy", ], )
              metrics=["binary_crossentropy", "auc"], )

pred_ans = model.predict(test_model_input, batch_size*20)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

for epoch in range(epochs):
    print('epoch',epoch)
    model.fit(train_model_input, train[target].values,
              batch_size=batch_size, epochs=1, validation_data=(val_model_input,val[target]), verbose=1, )

    pred_ans = model.predict(test_model_input, batch_size*20)
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.6932
test AUC 0.5156
epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 9460121 samples, validate on 774150 samples, 740 steps per epoch


740it [04:39,  2.65it/s]


Epoch 1/1
289s - loss:  0.3903 - binary_crossentropy:  0.3903 - auc:  0.7510 - val_binary_crossentropy:  0.4249 - val_auc:  0.7276
test LogLoss 0.4239
test AUC 0.7278
epoch 1


...

Epoch 1/1
280s - loss:  0.3417 - binary_crossentropy:  0.3417 - auc:  0.8204 - val_binary_crossentropy:  0.4436 - val_auc:  0.7073
test LogLoss 0.4427
test AUC 0.7074
epoch 4


0it [00:00, ?it/s]

cuda:0
Train on 9460121 samples, validate on 774150 samples, 740 steps per epoch


531it [03:17,  2.69it/s]


KeyboardInterrupt: 

In [8]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model_xfm = xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, dnn_dropout=0.5,
                task='binary', device=device)
print('model', model_xfm)

cuda ready...
model xDeepFM(
  (embedding_dict): ModuleDict(
    (C1): Embedding(7, 8)
    (C14): Embedding(2626, 8)
    (C15): Embedding(8, 8)
    (C16): Embedding(9, 8)
    (C17): Embedding(435, 8)
    (C18): Embedding(4, 8)
    (C19): Embedding(68, 8)
    (C20): Embedding(172, 8)
    (C21): Embedding(60, 8)
    (app_category): Embedding(36, 8)
    (app_domain): Embedding(559, 8)
    (app_id): Embedding(8552, 8)
    (banner_pos): Embedding(7, 8)
    (device_conn_type): Embedding(4, 8)
    (device_id): Embedding(2686408, 8)
    (device_model): Embedding(8251, 8)
    (device_type): Embedding(5, 8)
    (hour): Embedding(24, 8)
    (site_category): Embedding(26, 8)
    (site_domain): Embedding(7745, 8)
    (site_id): Embedding(4737, 8)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (C1): Embedding(7, 1)
      (C14): Embedding(2626, 1)
      (C15): Embedding(8, 1)
      (C16): Embedding(9, 1)
      (C17): Embedding(435, 1)
      (C18): Embedding(4, 1)
      (C19): E

In [9]:
all_metrics['xDeepFM'] = test_model(model_xfm, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size*2)

epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 1429 steps per epoch


1429it [23:42,  1.00it/s]


Epoch 1/1
1428s - loss:  0.3906 - binary_crossentropy:  0.3906 - auc:  0.7581 - val_binary_crossentropy:  0.4171 - val_auc:  0.7405


RuntimeError: CUDA out of memory. Tried to allocate 41.02 GiB (GPU 0; 15.78 GiB total capacity; 4.96 GiB already allocated; 9.60 GiB free; 230.85 MiB cached)

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model_dcn = DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, dnn_dropout=0.5,
                task='binary', device=device)

In [10]:
print('model', model_dcn)

model DCN(
  (embedding_dict): ModuleDict(
    (C1): Embedding(7, 8)
    (C14): Embedding(2626, 8)
    (C15): Embedding(8, 8)
    (C16): Embedding(9, 8)
    (C17): Embedding(435, 8)
    (C18): Embedding(4, 8)
    (C19): Embedding(68, 8)
    (C20): Embedding(172, 8)
    (C21): Embedding(60, 8)
    (app_category): Embedding(36, 8)
    (app_domain): Embedding(559, 8)
    (app_id): Embedding(8552, 8)
    (banner_pos): Embedding(7, 8)
    (device_conn_type): Embedding(4, 8)
    (device_id): Embedding(2686408, 8)
    (device_model): Embedding(8251, 8)
    (device_type): Embedding(5, 8)
    (hour): Embedding(24, 8)
    (site_category): Embedding(26, 8)
    (site_domain): Embedding(7745, 8)
    (site_id): Embedding(4737, 8)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (C1): Embedding(7, 1)
      (C14): Embedding(2626, 1)
      (C15): Embedding(8, 1)
      (C16): Embedding(9, 1)
      (C17): Embedding(435, 1)
      (C18): Embedding(4, 1)
      (C19): Embedding(68, 1)
  

In [11]:
torch.cuda.empty_cache()

In [12]:
del model_xfm

NameError: name 'model_xfm' is not defined

In [14]:
all_metrics['DCN'] = test_model(model_dcn, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size)

epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [20:55,  2.28it/s]


Epoch 1/1
1263s - loss:  0.3819 - binary_crossentropy:  0.3819 - auc:  0.7726 - val_binary_crossentropy:  0.4175 - val_auc:  0.7409
test LogLoss 0.4163
test AUC 0.7413


/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleDict. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + 

In [15]:
torch.cuda.empty_cache()

In [16]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model_dcn = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, dnn_dropout=0.5,
                task='binary', device=device)

cuda ready...


In [17]:
all_metrics['DeepFM'] = test_model(model_dcn, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size)


epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [20:30,  2.32it/s]


Epoch 1/1
1238s - loss:  0.3903 - binary_crossentropy:  0.3903 - auc:  0.7593 - val_binary_crossentropy:  0.4168 - val_auc:  0.7407
test LogLoss 0.4157
test AUC 0.741


/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DeepFM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleDict. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type "

In [18]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = FiBiNET(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                bilinear_type='all',
                reduction_ratio=8, dnn_hidden_units=(400, 400, 400), l2_reg_linear=1e-4,
                l2_reg_embedding=1e-4, l2_reg_dnn=1e-4, init_std=1e-15, seed=1024, dnn_dropout=0.3,
                dnn_activation='relu',
                task='binary', device=device)
print('model', model)

cuda ready...
model FiBiNET(
  (embedding_dict): ModuleDict(
    (C1): Embedding(7, 8)
    (C14): Embedding(2626, 8)
    (C15): Embedding(8, 8)
    (C16): Embedding(9, 8)
    (C17): Embedding(435, 8)
    (C18): Embedding(4, 8)
    (C19): Embedding(68, 8)
    (C20): Embedding(172, 8)
    (C21): Embedding(60, 8)
    (app_category): Embedding(36, 8)
    (app_domain): Embedding(559, 8)
    (app_id): Embedding(8552, 8)
    (banner_pos): Embedding(7, 8)
    (device_conn_type): Embedding(4, 8)
    (device_id): Embedding(2686408, 8)
    (device_model): Embedding(8251, 8)
    (device_type): Embedding(5, 8)
    (hour): Embedding(24, 8)
    (site_category): Embedding(26, 8)
    (site_domain): Embedding(7745, 8)
    (site_id): Embedding(4737, 8)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (C1): Embedding(7, 1)
      (C14): Embedding(2626, 1)
      (C15): Embedding(8, 1)
      (C16): Embedding(9, 1)
      (C17): Embedding(435, 1)
      (C18): Embedding(4, 1)
      (C19): E

In [19]:
torch.cuda.empty_cache()

In [20]:
all_metrics['FibiNet-all'] = test_model(model, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size)

epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [25:54,  1.84it/s]


Epoch 1/1
1562s - loss:  0.3879 - binary_crossentropy:  0.3878 - auc:  0.7633 - val_binary_crossentropy:  0.4160 - val_auc:  0.7436
test LogLoss 0.4148
test AUC 0.7441


/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FiBiNET. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleDict. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type 

In [21]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = FiBiNET(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                bilinear_type='interaction',
                reduction_ratio=8, dnn_hidden_units=(400, 400, 400), l2_reg_linear=1e-4,
                l2_reg_embedding=1e-4, l2_reg_dnn=1e-4, init_std=1e-15, seed=1024, dnn_dropout=0.3,
                dnn_activation='relu',
                task='binary', device=device)
print('model', model)

cuda ready...
model FiBiNET(
  (embedding_dict): ModuleDict(
    (C1): Embedding(7, 8)
    (C14): Embedding(2626, 8)
    (C15): Embedding(8, 8)
    (C16): Embedding(9, 8)
    (C17): Embedding(435, 8)
    (C18): Embedding(4, 8)
    (C19): Embedding(68, 8)
    (C20): Embedding(172, 8)
    (C21): Embedding(60, 8)
    (app_category): Embedding(36, 8)
    (app_domain): Embedding(559, 8)
    (app_id): Embedding(8552, 8)
    (banner_pos): Embedding(7, 8)
    (device_conn_type): Embedding(4, 8)
    (device_id): Embedding(2686408, 8)
    (device_model): Embedding(8251, 8)
    (device_type): Embedding(5, 8)
    (hour): Embedding(24, 8)
    (site_category): Embedding(26, 8)
    (site_domain): Embedding(7745, 8)
    (site_id): Embedding(4737, 8)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (C1): Embedding(7, 1)
      (C14): Embedding(2626, 1)
      (C15): Embedding(8, 1)
      (C16): Embedding(9, 1)
      (C17): Embedding(435, 1)
      (C18): Embedding(4, 1)
      (C19): E

In [22]:
torch.cuda.empty_cache()

In [23]:
all_metrics['FibiNet-inter'] = test_model(model, train_model_input, test_model_input, val_model_input, train[target].values, test[target].values, val[target], batch_size)

epoch 0


0it [00:00, ?it/s]

cuda:0
Train on 36558215 samples, validate on 774150 samples, 2857 steps per epoch


2857it [26:21,  1.81it/s]


Epoch 1/1
1589s - loss:  0.3881 - binary_crossentropy:  0.3881 - auc:  0.7629 - val_binary_crossentropy:  0.4163 - val_auc:  0.7434
test LogLoss 0.4151
test AUC 0.7437


/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FiBiNET. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleDict. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/conda/envs/espnet_env/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type 

In [29]:
get_metrics(all_metrics)

,DCN,DeepFM,FibiNet-all,FibiNet-inter,FibiNet-each,xDeepFM
metric,,,,,,
roc-auc,0.7413,0.7410,0.7441,0.7437,0.7437,0.7405
log-loss,0.4163,0.4157,0.4148,0.4151,0.4147,0.4171


In [23]:
get_ate(all_metrics, control_name='FibiNet-all')

,DCN,DeepFM,FibiNet-inter,FibiNet-each,xDeepFM
metric,,,,,
roc-auc,-0.28,-0.31,-0.04,-0.04,-0.36
log-loss,0.15,0.09,0.03,-0.01,0.23


In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
clf = LogisticRegression()

In [6]:
train[feature_names].head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,2,0,582,7339,2,7884,254,0,1780272,...,1,1,285,3,2,56,0,2,0,22
1,0,2,0,582,7339,2,7884,254,0,1780272,...,1,0,283,3,2,56,0,2,65,22
2,0,2,0,582,7339,2,7884,254,0,1780272,...,1,0,283,3,2,56,0,2,65,22
3,0,2,0,582,7339,2,7884,254,0,1780272,...,1,0,285,3,2,56,0,2,65,22
4,0,2,1,4695,4456,0,7884,254,0,1780272,...,1,0,590,3,2,107,0,2,0,42


In [11]:
clf.fit(train[feature_names], train[target].values)

/root/conda/envs/espnet_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [12]:
round(log_loss(test[target].values, clf.predict(test[feature_names])), 4)

6.2875

In [13]:
round(roc_auc_score(test[target].values, clf.predict(test[feature_names])), 4)

0.5